In [1]:
import json
import requests
import re
import nltk
import os 
from dotenv import load_dotenv
from nltk.tag import pos_tag
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from rake_nltk import Rake
#nltk.download('averaged_perceptron_tagger')
# nltk.download('stopwords')
# nltk.download('punkt')
# nltk.download('vader_lexicon')

In [2]:
def searchTweets(search_term:str):
    load_dotenv()
    url = "https://api.twitter.com/2/tweets/search/recent?max_results=25&expansions=referenced_tweets.id&tweet.fields=text&query=lang%3Aen%20"+search_term

    payload={}
    headers = {
    'Authorization': os.getenv('BEARER_TOKEN'),
    'Cookie': 'guest_id=v1%3A165292464833657250; guest_id_ads=v1%3A165292464833657250; guest_id_marketing=v1%3A165292464833657250; personalization_id="v1_D50leSEsdlQN9nTvwQ6B+g=="'
    }
    response = requests.request("GET", url, headers=headers, data=payload).json()['includes']['tweets']

    tweets = [i['text'] for i in response]


    # out_file = open("myfile.json", "w") 
    # json.dump(tweets, out_file, indent = 6) 
    # out_file.close() 
    
    # with open('myfile.json') as f:
    #  data = json.load(f)

    return tweets

In [3]:
tweets = searchTweets("Biden")

In [4]:
print(tweets)

['It was considered a huge offense when Trump did ‘mean tweets’ yet Biden can offend more than half the country with his fascist discourse &amp; it’s considered justifiable. \nThe double standard is beyond incomprehensible.', 'Has Biden cured cancer yet?', '“Why didn’t Joe Biden unify us??” https://t.co/eeS85fvQvX', 'I can׳t believe ABC, CBS, and NBC all declined to carry President Biden’s speech last night because it was too “political.”\n\nI’m too old to remember how they carried every Trump rant since the day he announced his presidential run until his last day in the WH. Shameful!', 'Biden admin’s push for a G7 price cap on Russian oil was strategic thinking aimed at closing off revenue streams for Putin to finance the war in Ukraine — https://t.co/i29IE2KKa1', 'So Elon’s $44B to buy twitter could have ended world hunger but nobody is asking about the $55B+ Biden has sent to ukraine?', 'Biden is the worst President in history. America is going to hell. Crimes have skyrocketed. Bide

In [5]:
#Build text pre-processing 
#would a class for the tweets be smart?

### Text Preprocessing

In [6]:

def process_text(text):
    stop_words = set(stopwords.words('english'))
    clean_text = re.sub(r'\b(https?://\S+)\b','',text) #remove URLs
    clean_text = " ".join(re.findall('\w{3,}',clean_text))
    word_tokens = word_tokenize(clean_text)
    filtered_text = " ".join([word for word in word_tokens if word not in stop_words])
    
    return filtered_text

In [7]:
clean_tweets = [process_text(i) for i in tweets]

In [8]:
#get keywords
clean_tweets

['considered huge offense Trump mean tweets yet Biden offend half country fascist discourse amp considered justifiable The double standard beyond incomprehensible',
 'Has Biden cured cancer yet',
 'Why Joe Biden unify',
 'believe ABC CBS NBC declined carry President Biden speech last night political old remember carried every Trump rant since day announced presidential run last day Shameful',
 'Biden admin push price cap Russian oil strategic thinking aimed closing revenue streams Putin finance war Ukraine',
 'Elon 44B buy twitter could ended world hunger nobody asking 55B Biden sent ukraine',
 'Biden worst President history America going hell Crimes skyrocketed Biden letting many terrorists around world You believe many allahu akbar incidents corrupt media hides Biden loves Pakistan USA',
 'Please retweet help NikkiHaley understand Joe Biden calls MAGA Republicans bad people',
 'RyanAFournier False million stood line voted Biden rally really counted You welcome',
 'Biden calling half 

In [9]:
from statistics import mean
from math import floor

def get_keywords(sentences: list):
    '''
    Given a list of strings, return the top keywords that are at least 4 letters long
    '''        
    r = Rake()
    r.extract_keywords_from_sentences(sentences)
    r.get_ranked_phrases_with_scores()
    keywords = r.get_ranked_phrases()[0].split(" ")
    avg_keyword_len = floor(mean([len(i) for i in keywords]))
    best_keywords = [i for i in keywords if len(i)>avg_keyword_len and len(i)>3]
    return best_keywords

In [10]:
get_keywords(clean_tweets)


['believe',
 'declined',
 'president',
 'speech',
 'political',
 'remember',
 'carried',
 'announced',
 'presidential',
 'shameful']

In [11]:
#get sentiment
from nltk.sentiment import SentimentIntensityAnalyzer
def get_avg_sentiment(tweets):
    sia = SentimentIntensityAnalyzer()

    term_sentiment = round(mean([sia.polarity_scores(tweet)['compound'] for tweet in tweets ]),2)

    if term_sentiment < -0.33:
        return ("negative", term_sentiment)
    elif term_sentiment > 0.33:
        return ("positive", term_sentiment)
    else:
        return ("neutral",term_sentiment)

get_avg_sentiment(clean_tweets)


('neutral', -0.22)

In [12]:
keywords = get_keywords(clean_tweets,)

tag=pos_tag(keywords)

[i[0] for i in tag if i[1] not in ['VBG','VBD','RB','PDT','POS','WDT','WP','WRB',]]

['believe',
 'president',
 'speech',
 'political',
 'remember',
 'announced',
 'presidential',
 'shameful']

In [13]:
def call_trends():
  url = "https://api.twitter.com/1.1/trends/place.json?id=23424977"
  headers = {
  'Authorization': os.getenv("BEARER_TOKEN"),
  'Cookie': 'guest_id=v1%3A165292464833657250; guest_id_ads=v1%3A165292464833657250; guest_id_marketing=v1%3A165292464833657250; personalization_id="v1_D50leSEsdlQN9nTvwQ6B+g=="'
  }

  try:
    response = requests.request("GET",url,headers=headers).json()[0]['trends']
    response.sort(key=lambda x:0 if x["tweet_volume"] is None else x["tweet_volume"],reverse=True)
    results = [trend['name'] for trend in response]
    return results
  except Exception as e:
    error_message = f"An error occured in call_trends(): {e}"
    return error_message

In [14]:
x = call_trends()
print(x)

['Bam Bam', '#WWECastle', '#UFCParis', 'Roman', 'Gane', 'Gane', 'Drew', '#taylorhawkinstribute', 'Wilbur', 'Utah', 'Oregon', 'Oregon', 'Iowa', 'Arizona', 'My Hero', 'Seth', 'Gunther', 'Sheamus', 'Arkansas', 'App State', 'Dave Grohl', '#GoDawgs', 'Whittaker', 'Bo Nix', 'Broken Dreams', 'Tyson Fury', 'Buckley', 'Vettori', 'NC State', 'Cincinnati', '#OREvsUGA', 'Stetson Bennett', 'North Dakota', 'Bobby Knuckles', 'Anthony Grant', 'UTSA', 'Auburn', 'Ladd McConkey', 'PAC 12', 'Dan Lanning', 'Shane Hawkins', 'UTEP', 'UCLA', 'SDSU', 'Ty Thompson', 'Cincy', 'Ben Bryant', 'East Carolina', 'Darnell Washington', 'Caleb Williams']


In [31]:
def get_results(term):
	try:	
		if term:
			raw_results = searchTweets(term)
			processed_results = [process_text(tweet) for tweet in raw_results]

			sentiment = get_avg_sentiment(processed_results)	
			keywords = get_keywords(processed_results)

			results = {"sentiment" : sentiment[0],
				"score": sentiment[1],
				"keywords":keywords
			}
			return {"sentiment" : results}
		else:
			return {"message":"nothing to seach"}
	except Exception as e:
		return {"error":e}

In [32]:
results = get_results("Linux")['sentiment']

In [33]:
results['sentiment']

'neutral'

In [35]:
results

{'sentiment': 'neutral',
 'score': 0.08,
 'keywords': ['dataanalyst',
  'machinelearning',
  'datascience',
  'cybersecurity',
  'analytics',
  'tensorflow',
  'javascript',
  'cloudcomputing',
  'serverless',
  'datascientist',
  'programming',
  '100daysofcode']}

In [64]:
import plotly.graph_objects as go
    
fig = go.Figure(go.Indicator(
    mode = "gauge+number",
    value = 0.5,
    domain = {'x': [0, 1], 'y': [0, 1]},
    title = {'text': "Sentiment"},
    gauge = {
        'axis': {'range': [-1, 1], 'tickwidth': 1, 'tickcolor': "darkblue"},
        'bar': {'color': "darkblue"},
        'bgcolor': "white",
        'borderwidth': 2,
        'bordercolor': "gray",
        'steps': [
            {'range': [-1, -0.10], 'color': 'red'},
            {'range': [-.10, 0.10], 'color': 'lightgrey'},
            {'range': [0.10, 1], 'color': 'green'}],
        'threshold': {
            'line': {'color': "red", 'width': 4},
            'thickness': 0.5,
            'value': 0.5}}))

fig.write_html('app/templates/score_gauge_test.html')


'<div>                            <div id="bef00c3d-86c3-462c-8dcc-39ec7a659186" class="plotly-graph-div" style="height:100%; width:100%;"></div>            <script type="text/javascript">                                    window.PLOTLYENV=window.PLOTLYENV || {};                                    if (document.getElementById("bef00c3d-86c3-462c-8dcc-39ec7a659186")) {                    Plotly.newPlot(                        "bef00c3d-86c3-462c-8dcc-39ec7a659186",                        [{"domain":{"x":[0,1],"y":[0,1]},"mode":"gauge+number","title":{"text":"Sentiment"},"value":0.08,"type":"indicator"}],                        {"template":{"data":{"barpolar":[{"marker":{"line":{"color":"#E5ECF6","width":0.5},"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"barpolar"}],"bar":[{"error_x":{"color":"#2a3f5f"},"error_y":{"color":"#2a3f5f"},"marker":{"line":{"color":"#E5ECF6","width":0.5},"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"bar"}],"carpet":[{"a

In [47]:
x

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'domain': {'x': [0, 1], 'y': [0, 1]},
              'mode': 'gauge+number',
              'title': {'text': 'Sentiment'},
              'type': 'indicator',
              'value': 0.08}],
    'layout': {'template': '...'}
})